In [ ]:
#Clean notebook for data clean up explantions

In [ ]:
#imported our Dependancies as needed
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import requests
import json
import datetime as dt

#Importing CSV files found online
state_lockdown_file = pd.read_csv("data_sources/lockdown_us.csv")
state_deaths_cases_file = pd.read_csv("data_sources/liz-friedman-us-covid-19-data-from-nytimes/us-states.csv")
county_population_file = pd.read_csv("data_sources/qventus-covid-19-localized-scenario-planner/qventus-covid-19-localized-scenario-planner/covid_county_population_usafacts.csv")

In [ ]:
#URL pulls state data just for the current or last weekday reported
url = "https://api.covidtracking.com/v1/states/current.json"
state_current_data = (requests.get(url)).json()
#Creating dataframe form api request
state_current_data=pd.DataFrame(state_current_data)
#date format is yyyymmdd this line changes to yyyy-mm-dd
state_current_data['date'] = pd.to_datetime(state_current_data['date'], format='%Y%m%d')

In [ ]:
#URL pulls collection of all states current data from past dates
url = "https://api.covidtracking.com/v1/states/daily.json"
state_daily_data = (requests.get(url)).json()

#Creating dataframe form api request
state_daily_data=pd.DataFrame(state_daily_data)

#date format is yyyymmdd this line changes to yyyy-mm-dd
state_daily_data['date'] = pd.to_datetime(state_daily_data['date'], format='%Y%m%d')

#Dropping US Territories from state list
state_daily_data = state_daily_data.set_index("state")
state_daily_data = state_daily_data.drop(["AS","GU","MP","PR", "VI","DC"])
state_daily_data = state_daily_data.reset_index()

#loop for summing for total state cases
states = state_daily_data["state"].unique()
total_cases_by_state = []

for state in states:
    state_case = state_daily_data.loc[state_daily_data["state"] == state]
    total_cases_by_state.append(state_case["positive"].sum())


In [ ]:
#loop for finding positive rate per month

months = [1,2,3,4,5,6,7,8]
positive_rates = []
state_list = []
month_list = []

for state in states:
    
    single_state = state_daily_data.loc[state_daily_data["state"] == state]
    single_state['date'] = pd.to_datetime(single_state['date'], format='%Y%m%d')    
    single_state = single_state.sort_values("date")
    single_state['month'] = pd.DatetimeIndex(single_state['date']).month
    single_state['day'] = pd.DatetimeIndex(single_state['date']).day
    for month in months:
        print(state)
        try:
            single_month = single_state.loc[single_state["month"] == month]
            single_month = single_month.reset_index()
            first_row = single_month.iloc[0]
            last_row = single_month.iloc[-1]
            month_pos = int(last_row["positive"]) - int(first_row["positive"])
            month_tot = int(last_row["posNeg"]) - int(first_row["posNeg"])
            positive_rate = month_pos / month_tot
            positive_rate = "{:.2%}".format(positive_rate)
            positive_rates.append(positive_rate)
            state_list.append(state)
            month_list.append(month)
            
            
        except IndexError:
            #some states had no cases in the first 2 months
            print("no data for this month")
        except ValueError:
            #New Jersey had had values of NaN causing a value error
            print("value issue")
        except ZeroDivisionError:
            #Washington first month total comes out to 0 causing a division by zero error
            print("month total = 0")

In [ ]:
#Dataframe created from above loop
pos_rates_df = pd.DataFrame({
    "State": state_list,
    "Positive Rate": positive_rates,
    "Month": month_list
})

In [ ]:
#States total population by summing all counties population
state_pop = []
states = county_population_file["State"].unique()

for state in states:
    state_df = county_population_file.loc[county_population_file["State"] == state]
    state_pop.append(state_df["population"].sum())
    
population_df = pd.DataFrame({
    "state": states,
    "population": state_pop
})